# Trabalho Final de IA
Aluno: André Valente de Cristo

Trabalho feito e testado no Google Colab.

## Instalação de pacotes

In [1]:
!pip install pandas_ods_reader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49002 sha256=38013e2ab0db1596c64864b2bf53d6db6c7ccc1566f7f5a8b801cf8255184892
  Stored in directory: /root/.cache/pip/wheels/50/09/62/ea2c44e6ffdd067d6c8d3f557af0734a195bd252d6901f23ca
Successfully built ezodf


In [2]:
pip install LTNtorch

## Imports

In [3]:
#Basic imports
from google.colab import drive
import os
from pandas_ods_reader import read_ods
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn imports
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#ltn imports
import ltn
import torch


# Base de dados

Neste trabalho foi usada a base de 100 trens do East-West Challenge do Oxford University Computing Laboratory, estabelecida no arquivo 100_trains_set.pdf e trains-updated.csv.

Para garantir a integridade da base, uma verififcação aleatória foi feita, comparando tuplas do arquivo .csv com as imagens dos trens no arquivo .pdf. Todos os exemplos verificados estavam consistentes.

## Carregamento da base

Durante o desenvolvimento, a base foi carregada a partir do Google Drive, para facilitar o uso com o Google Colab. Também é possível carregá-la através do arquivo trains-uptated.csv no diretório do notebook. Se o teste estiver sendo feito no Google Colab, ainda será necessário fazer upload do arquivo para o diretório do Colab.

In [5]:
# drive.flush_and_unmount()

In [6]:
# drive.mount('/content/drive')

In [7]:
sns.set()
# base_path = "/content/drive/MyDrive/IA_TrabFinal/trains-uptated.csv"
base_path = "trains-uptated.csv"

data = pd.read_csv(base_path)

data

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,...,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute
0,4,2.0,2,long,openrect,1.0,rectanglod,2,short,openrect,...,1,0,0,0,0,0,0,0,0,east
1,4,2.0,2,short,openrect,2.0,circlelod,2,short,closedtrap,...,0,0,1,0,0,0,0,0,0,east
2,4,3.0,2,short,ushaped,1.0,trianglod,2,short,openrect,...,1,0,1,0,0,0,0,0,0,east
3,5,3.0,2,short,opentrap,1.0,circlelod,2,short,ushaped,...,1,0,0,0,0,1,0,0,0,east
4,4,3.0,2,short,closedrect,1.0,trianglod,2,short,closedtrap,...,0,0,1,0,0,1,0,0,0,east
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,3.0,2,short,openrect,3.0,rectanglod,2,short,hexagon,...,0,0,1,1,0,1,0,0,0,west
96,3,2.0,2,short,hexagon,2.0,circlelod,2,long,closedrect,...,0,0,1,0,0,0,0,0,0,west
97,3,2.0,2,short,slopetoprect,1.0,circlelod,3,long,closedrect,...,0,0,0,1,0,0,0,0,0,west
98,3,1.0,2,short,slopetoprect,1.0,trianglod,2,short,openrect,...,0,0,0,1,0,0,0,0,0,west


In [8]:
data.columns

Index(['Number_of_cars', 'Number_of_different_loads', 'num_wheels1', 'length1',
       'shape1', 'num_loads1', 'load_shape1', 'num_wheels2', 'length2',
       'shape2', 'num_loads2', 'load_shape2', 'num_wheels3', 'length3',
       'shape3', 'num_loads3', 'load_shape3', 'num_wheels4', 'length4',
       'shape4', 'num_loads4', 'load_shape4', 'Rectangle_next_to_rectangle',
       'Rectangle_next_to_triangle', 'Rectangle_next_to_hexagon',
       'Rectangle_next_to_circle', 'Triangle_next_to_triangle',
       'Triangle_next_to_hexagon', 'Triangle_next_to_circle',
       'Hexagon_next_to_hexagon', 'Hexagon_next_to_circle',
       'Circle_next_to_circle', 'Class_attribute'],
      dtype='object')

In [9]:
for column in data.columns:
  print(f'{column}: {data[column].unique()}\n')

Number_of_cars: [4 5 3]

Number_of_different_loads: [ 2.  3.  1.  4. nan]

num_wheels1: [2 3]

length1: ['long' 'short']

shape1: ['openrect' 'ushaped' 'opentrap' 'closedrect' 'closedblopnrect' 'ellipse'
 'slopetopdblopnrect' 'slopetoprect' 'dblopnrect' 'slopetoptrap'
 'closedushaped' 'jaggedrect' 'hexagon' 'closedtrap']

num_loads1: [ 1.  2. nan  3.  0.]

load_shape1: ['rectanglod' 'circlelod' 'trianglod' nan 'hexagonlod']

num_wheels2: [2 3]

length2: ['short' 'long']

shape2: ['openrect' 'closedtrap' 'ushaped' 'opentrap' 'ellipse' 'closedrect'
 'dblopnrect' 'hexagon' 'jaggedrect' 'slopetoprect']

num_loads2: [ 1.  3.  2. nan]

load_shape2: ['trianglod' 'rectanglod' 'circlelod' 'hexagonlod' nan]

num_wheels3: [ 2. nan  3.]

length3: ['long' 'short' nan]

shape3: ['closedrect' 'openrect' 'ushaped' nan 'closedtrap' 'opentrap'
 'closedblopnrect' 'ellipse' 'slopetoprect' 'hexagon' 'jaggedrect'
 'dblopnrect' 'slopetoptrap']

num_loads3: [ 3.  1. nan  2.]

load_shape3: ['rectanglod' 'circl

## Removendo NaNs

Vários valores na base contém NaNs, geralmente representando valores em carros que não existem. Embora hajam algumas excessões, como Number_of_different_loads ser igual a NaN quando o trem não tem carros. Em todos os casos o NaN pode ser substituido por 0.

Adicionalmente, em um trem a string 'none' é usada no num_wheels4 ao invés de NaN. Esse caso também deve ser substituido por 0.

In [10]:
# removendo o none
data.loc[data['num_wheels4'] == 'none', 'num_wheels4'] = 0

# substituindo os NaNs por 0
data = data.fillna(0)

In [11]:
for column in data.columns:
  print(f'{column}: {data[column].unique()}\n')

Number_of_cars: [4 5 3]

Number_of_different_loads: [2. 3. 1. 4. 0.]

num_wheels1: [2 3]

length1: ['long' 'short']

shape1: ['openrect' 'ushaped' 'opentrap' 'closedrect' 'closedblopnrect' 'ellipse'
 'slopetopdblopnrect' 'slopetoprect' 'dblopnrect' 'slopetoptrap'
 'closedushaped' 'jaggedrect' 'hexagon' 'closedtrap']

num_loads1: [1. 2. 0. 3.]

load_shape1: ['rectanglod' 'circlelod' 'trianglod' 0 'hexagonlod']

num_wheels2: [2 3]

length2: ['short' 'long']

shape2: ['openrect' 'closedtrap' 'ushaped' 'opentrap' 'ellipse' 'closedrect'
 'dblopnrect' 'hexagon' 'jaggedrect' 'slopetoprect']

num_loads2: [1. 3. 2. 0.]

load_shape2: ['trianglod' 'rectanglod' 'circlelod' 'hexagonlod' 0]

num_wheels3: [2. 0. 3.]

length3: ['long' 'short' 0]

shape3: ['closedrect' 'openrect' 'ushaped' 0 'closedtrap' 'opentrap'
 'closedblopnrect' 'ellipse' 'slopetoprect' 'hexagon' 'jaggedrect'
 'dblopnrect' 'slopetoptrap']

num_loads3: [3. 1. 0. 2.]

load_shape3: ['rectanglod' 'circlelod' 0 'trianglod' 'hexagonlod'

## Codificação da base

Como os dados da base estão escritos por extenso, é necessário codificá-la, passando-os para valores numéricos que podem ser inseridos nos modelos. Para isso foi feito uso dos códigos informados em trains-data_coded.csv.

In [12]:
dict_length = {
    'short': 1,
    'long': 2
}

dict_shape = {
    'closedblopnrect': 1,
    'closedrect': 2,
    'closedtrap': 3,
    'closedushaped': 4,
    'dblopnrect': 5,
    'ellipse': 6,
    'hexagon': 7,
    'jaggedrect': 8,
    'openrect': 9,
    'opentrap': 10,
    'slopetopdblopnrect': 11,
    'slopetoprect': 12,
    'slopetoptrap': 13,
    'slopetopushaped': 14,
    'ushaped': 15
}

dict_loadshape = {
    'circlelod': 1,
    'hexagonlod': 2,
    'rectanglod': 3,
    'trianglod': 4
}

dict_goingto = {
    'west': 0,
    'east': 1
}

In [13]:
data.replace(dict_length, inplace=True)
data.replace(dict_shape, inplace=True)
data.replace(dict_loadshape, inplace=True)
data.replace(dict_goingto, inplace=True)

<ipython-input-13-83dc84257859>:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-13-83dc84257859>:2: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-13-83dc84257859>:3: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-13-83dc84257859>:4: FutureWarning:

Downcastin

In [14]:
for column in data.columns:
    data[column] = data[column].astype(int)

In [15]:
for column in data.columns:
    print(f'{column}: {data[column].unique()}\n')

Number_of_cars: [4 5 3]

Number_of_different_loads: [2 3 1 4 0]

num_wheels1: [2 3]

length1: [2 1]

shape1: [ 9 15 10  2  1  6 11 12  5 13  4  8  7  3]

num_loads1: [1 2 0 3]

load_shape1: [3 1 4 0 2]

num_wheels2: [2 3]

length2: [1 2]

shape2: [ 9  3 15 10  6  2  5  7  8 12]

num_loads2: [1 3 2 0]

load_shape2: [4 3 1 2 0]

num_wheels3: [2 0 3]

length3: [2 1 0]

shape3: [ 2  9 15  0  3 10  1  6 12  7  8  5 13]

num_loads3: [3 1 0 2]

load_shape3: [3 1 0 4 2]

num_wheels4: [0 2 3]

length4: [0 1 2]

shape4: [ 0  9  2 15  8 10  3  5  6  7  4 14]

num_loads4: [0 1 2 3]

load_shape4: [0 3 4 1 2]

Rectangle_next_to_rectangle: [0 1]

Rectangle_next_to_triangle: [1 0]

Rectangle_next_to_hexagon: [0 1]

Rectangle_next_to_circle: [0 1]

Triangle_next_to_triangle: [0 1]

Triangle_next_to_hexagon: [0 1]

Triangle_next_to_circle: [0 1]

Hexagon_next_to_hexagon: [0]

Hexagon_next_to_circle: [0 1]

Circle_next_to_circle: [0 1]

Class_attribute: [1 0]



# **Questão 1: Clusterização**

Para procurar similaridades no dataset, de forma a gerar axiomas para a predição da direção de viajem dos trens, foi feita a clusterização através do modelo K Means, fornecido pela biblioteca SKLearn. Para garantir a replicabilidade da clusterização e sua análise posterior, ela foi feita com a semente 123.

## Criação dos clusters

In [16]:
X = data.copy()
header = X.loc[0 ,:]

X

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,...,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute
0,4,2,2,2,9,1,3,2,1,9,...,1,0,0,0,0,0,0,0,0,1
1,4,2,2,1,9,2,1,2,1,3,...,0,0,1,0,0,0,0,0,0,1
2,4,3,2,1,15,1,4,2,1,9,...,1,0,1,0,0,0,0,0,0,1
3,5,3,2,1,10,1,1,2,1,15,...,1,0,0,0,0,1,0,0,0,1
4,4,3,2,1,2,1,4,2,1,3,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,3,2,1,9,3,3,2,1,7,...,0,0,1,1,0,1,0,0,0,0
96,3,2,2,1,7,2,1,2,2,2,...,0,0,1,0,0,0,0,0,0,0
97,3,2,2,1,12,1,1,3,2,2,...,0,0,0,1,0,0,0,0,0,0
98,3,1,2,1,12,1,4,2,1,9,...,0,0,0,1,0,0,0,0,0,0


In [17]:
# Lable removida para não ser considerada quando pelo k-means
X_noLabel = X.drop(["Class_attribute"], axis=1)
X_noLabel

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,...,Rectangle_next_to_rectangle,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle
0,4,2,2,2,9,1,3,2,1,9,...,0,1,0,0,0,0,0,0,0,0
1,4,2,2,1,9,2,1,2,1,3,...,0,0,0,1,0,0,0,0,0,0
2,4,3,2,1,15,1,4,2,1,9,...,0,1,0,1,0,0,0,0,0,0
3,5,3,2,1,10,1,1,2,1,15,...,1,1,0,0,0,0,1,0,0,0
4,4,3,2,1,2,1,4,2,1,3,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,3,2,1,9,3,3,2,1,7,...,0,0,0,1,1,0,1,0,0,0
96,3,2,2,1,7,2,1,2,2,2,...,0,0,0,1,0,0,0,0,0,0
97,3,2,2,1,12,1,1,3,2,2,...,0,0,0,0,1,0,0,0,0,0
98,3,1,2,1,12,1,4,2,1,9,...,0,0,0,0,1,0,0,0,0,0


In [18]:
kmeans = KMeans(n_clusters=2, random_state=123)
#Fit our model
kmeans.fit(X_noLabel)

KMeans(n_clusters=2, random_state=123)

In [19]:
clusters = kmeans.predict(X_noLabel)
#Add the cluster vector to our DataFrame, X
X["Cluster"] = clusters
X

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,...,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute,Cluster
0,4,2,2,2,9,1,3,2,1,9,...,0,0,0,0,0,0,0,0,1,0
1,4,2,2,1,9,2,1,2,1,3,...,0,1,0,0,0,0,0,0,1,1
2,4,3,2,1,15,1,4,2,1,9,...,0,1,0,0,0,0,0,0,1,0
3,5,3,2,1,10,1,1,2,1,15,...,0,0,0,0,1,0,0,0,1,1
4,4,3,2,1,2,1,4,2,1,3,...,0,1,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,3,2,1,9,3,3,2,1,7,...,0,1,1,0,1,0,0,0,0,1
96,3,2,2,1,7,2,1,2,2,2,...,0,1,0,0,0,0,0,0,0,0
97,3,2,2,1,12,1,1,3,2,2,...,0,0,1,0,0,0,0,0,0,0
98,3,1,2,1,12,1,4,2,1,9,...,0,0,1,0,0,0,0,0,0,0


## Vizualização dos clusters

Para ajudar na vizualização dos clusters, foi usada uma técnica de redução de dimensionalidade, baixando de 32 dimensões para 2, de forma a plotar os dados em um gráfico 2D.

In [20]:
#PCA with one principal component
pca_1d = PCA(n_components=1)

#PCA with two principal components
pca_2d = PCA(n_components=2)

In [21]:
#This DataFrame holds that single principal component mentioned above
PCs_1d = pd.DataFrame(pca_1d.fit_transform(X))

#This DataFrame contains the two principal components that will be used
#for the 2-D visualization mentioned above
PCs_2d = pd.DataFrame(pca_2d.fit_transform(X))

In [22]:
PCs_1d.columns = ["PC1_1d"]

#"PC1_2d" means: 'The first principal component of the components created for 2-D visualization, by PCA.'
#And "PC2_2d" means: 'The second principal component of the components created for 2-D visualization, by PCA.'
PCs_2d.columns = ["PC1_2d", "PC2_2d"]

In [23]:
plotClusters = []
plotClusters = [X["Cluster"], PCs_1d,PCs_2d]

plotX = pd.concat([X,PCs_1d,PCs_2d], axis=1, join='inner')
plotX["dummy"] = 0

X[X["Cluster"] == 0]

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,...,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute,Cluster
0,4,2,2,2,9,1,3,2,1,9,...,0,0,0,0,0,0,0,0,1,0
2,4,3,2,1,15,1,4,2,1,9,...,0,1,0,0,0,0,0,0,1,0
7,3,2,2,1,1,1,3,2,1,15,...,0,0,0,0,0,0,0,0,1,0
8,5,3,2,1,9,1,1,2,1,10,...,0,1,0,0,0,0,0,1,1,0
9,3,2,2,1,15,1,1,3,2,9,...,0,1,0,0,0,0,0,0,1,0
10,5,2,2,1,15,1,4,3,2,9,...,0,0,1,0,0,0,0,0,1,0
12,3,2,2,1,15,1,1,2,1,6,...,0,0,0,0,1,0,0,0,1,0
14,3,2,2,1,10,1,3,2,1,15,...,0,0,0,0,0,0,0,0,1,0
15,3,2,2,1,9,1,4,2,1,9,...,0,0,0,0,1,0,0,0,1,0
16,5,3,2,2,2,1,3,2,1,2,...,0,1,0,0,1,0,0,1,1,0


In [24]:
A = np.array(plotX["Cluster"])
fA= pd.DataFrame(A.astype(int))
fA = pd.concat([fA,PCs_2d], axis=1, join='inner')
fA

,0,PC1_2d,PC2_2d
0,0,-3.932227,-0.409748
1,1,-1.032793,4.542558
2,0,-4.242795,-1.766009
3,1,8.488415,-1.408355
4,1,3.110198,10.522069
...,...,...,...
95,1,5.566936,1.620884
96,0,-7.561382,-1.726044
97,0,-7.431960,-2.822628
98,0,-6.413959,-2.632017


In [25]:
trains_cluster = []
colors = ['rgba(255, 128, 255, 0.8)', 'rgba(255, 128, 2, 0.8)', 'rgba(0, 255, 200, 0.8)','rgba(255, 55, 0, 0.8)']

for i in range(len(fA)):
    trains_cluster.append(go.Scatter(
        x = [fA["PC1_2d"][i]],
        y = [fA["PC2_2d"][i]],
        mode = "markers",
        name = f'Train {i}',
        marker = dict(color = colors[int(fA[0][i])]),
    ))

title = "Visualizing Clusters in One Dimension Using PCA"

data = trains_cluster #, trace4]

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

## Busca de características dos clusters

Por fim, para gerar os possíveis axiomas, foi feita uma análise das características dos clusters, para entender o que levou cada trem a ser colocado no cluster em que está.

In [26]:
def split_dataset(df, column, margin):
    distinct_values = df[column].unique()

    list_df = []
    for value in distinct_values:
        filtered_df = df[np.abs(df[column] - value) <= margin]
        list_df.append(filtered_df)

    return list_df

In [27]:
def check_split(column, margin):
    print(f'column: {column}\n')

    for i in range(len(X['Cluster'].unique())):
        cluster = X[X["Cluster"] == i]
        df_num_cars = split_dataset(cluster, column, margin)

        print(f'Unique values in cluster {i}: {len(df_num_cars)}')

        for df in df_num_cars:
            print(f'{df[column].iloc[0]}: {len(df)}')

        print()

Como é possível ver na célula abaixo, os clusters não tem nenhuma divisão relevante em relação a direção em que irão. Portanto, qualquer lógica que seja encontrada neles não ajudará em encontrar a regra geral do problema.

In [28]:
check_split('Class_attribute', 0)

column: Class_attribute

Unique values in cluster 0: 2
1: 25
0: 26

Unique values in cluster 1: 2
1: 25
0: 24



Com isso dito, para encontrar as características, pode-se começar verificando como cada coluna foi dividida.

In [29]:
for column in X.columns:
    check_split(column, 0)

column: Number_of_cars

Unique values in cluster 0: 3
4: 12
3: 26
5: 13

Unique values in cluster 1: 2
4: 24
5: 25

column: Number_of_different_loads

Unique values in cluster 0: 4
2: 38
3: 7
1: 5
0: 1

Unique values in cluster 1: 4
2: 23
3: 19
1: 4
4: 3

column: num_wheels1

Unique values in cluster 0: 2
2: 49
3: 2

Unique values in cluster 1: 2
2: 46
3: 3

column: length1

Unique values in cluster 0: 2
2: 10
1: 41

Unique values in cluster 1: 2
1: 47
2: 2

column: shape1

Unique values in cluster 0: 12
9: 14
15: 8
1: 1
10: 7
2: 7
4: 2
8: 2
7: 2
5: 2
12: 3
13: 2
6: 1

Unique values in cluster 1: 12
9: 19
10: 8
2: 6
6: 3
11: 1
15: 3
12: 1
5: 4
13: 1
8: 1
3: 1
7: 1

column: num_loads1

Unique values in cluster 0: 4
1: 43
2: 3
3: 3
0: 2

Unique values in cluster 1: 4
2: 4
1: 43
0: 1
3: 1

column: load_shape1

Unique values in cluster 0: 5
3: 19
4: 13
1: 15
0: 2
2: 2

Unique values in cluster 1: 5
1: 20
4: 14
3: 13
0: 1
2: 1

column: num_wheels2

Unique values in cluster 0: 2
2: 41
3: 10


Todos os trens com 3 carros foram para o cluster 0, e a maior parte dos outros foram para o 1. Também há várias diferenças nas colunas referentes aos carros 3 e 4, mas isso é, de forma geral, resultado da diferença no número de carros.



In [30]:
check_split('Number_of_cars', 0)

column: Number_of_cars

Unique values in cluster 0: 3
4: 12
3: 26
5: 13

Unique values in cluster 1: 2
4: 24
5: 25



No entanto, também houveram outras divisões interessantes, como o formato do carro 3, onde todos os carros retangulares fechados (número 2) foram para o cluster 0, apesar deste ser o com menos carros 3.

In [31]:
check_split('shape3', 0)

column: shape3

Unique values in cluster 0: 5
2: 19
0: 26
3: 2
1: 3
5: 1

Unique values in cluster 1: 9
9: 20
15: 8
10: 5
6: 4
12: 2
7: 2
8: 6
13: 1
5: 1



Também houve um maior número de triângulos perto de círculos no cluster 1, embora foi menos significativo que o formato do carro 3.

In [32]:
check_split('Triangle_next_to_circle', 0)

column: Triangle_next_to_circle

Unique values in cluster 0: 2
0: 42
1: 9

Unique values in cluster 1: 2
0: 23
1: 26



Dadas essas características, alguns possíveis axiomas seriam:
 - "Para todo trem, se o trem tiver 3 carros, ele pertencerá ao cluster 0".
 - "Para todo trem, se existe um carro retangular fechado, o trem pertencerá ao cluster 0".
 - "Para todo trem, se existe um triângulo perto de círculo, o trem pertence ao cluster 1".

# **Questão 2: LTN**



Com os axiomas obtidos, pode ser criada uma LTN para prever se a direção de viagem dos trens. Para isso, foi usada a biblioteca LTNTorch, seguindo os tutoriais e exemplos disponibilizados na página do GitHub da biblioteca.

Não foi possível finalizar essa questão, devido a falta de tempo e familiaridade com o LTNTorch, mas a lógica que tentei implementar e uma implementação baseada no exemplo de classificação binária que funciona, mas não usa os predicados e axiomas da forma correta estão disponibilizadas neste notebook.

## Implementação incompleta

Para implementar a LTN com os predicados dados e os axiomas da questão 1, é necessário fazer uso de uma meta-rede, onde cada predicado corresponde a um modelo PyTorch. O agregador de satisfabilidade usado no treinamento então verificaria os axiomas, fazendo a back-propagation nos predicados relevantes.

### Preparo da base

Para montar os predicados, é necessário extrair as informações relevantes da base de dados. Por exemplo, a base guarda a informação do tamanho do carro 4 do trem 17 na coluna 'length4' da linha 17, então é necessário extrair a informação da linha (trem), do número do length da coluna (carro), e o valor da célula (o tamanho em si) para gerar o predicado *length(17, 4, 1)*.

In [49]:
# Process dataset to create arguments for predicates
def preprocess_dataset(X):
    """
    Convert the dataset into a format suitable for the predicates.
    Returns a dictionary where each key is a predicate name, and the value is the arguments for that predicate.
    """
    predicate_data = {
        "num_cars": [],
        "num_loads": [],
        "num_wheels": [],
        "length": [],
        "shape": [],
        "num_car_loads": [],
        "load_shape": [],
        "direction": []
    }

    for idx, row in X.iterrows():
        t = idx  # train index
        predicate_data["num_cars"].append([t, row["Number_of_cars"]])
        predicate_data["num_loads"].append([t, row["Number_of_different_loads"]])
        for car_num in range(1, int(row["Number_of_cars"])):
            predicate_data["num_wheels"].append([t, car_num, row[f"num_wheels{car_num}"]])
            predicate_data["length"].append([t, car_num, row[f"length{car_num}"]])
            predicate_data["shape"].append([t, car_num, row[f"shape{car_num}"]])
            predicate_data["num_car_loads"].append([t, car_num, row[f"num_loads{car_num}"]])
            predicate_data["load_shape"].append([t, car_num, row[f"load_shape{car_num}"]])
        predicate_data["direction"].append([t, row["Class_attribute"]])

    return predicate_data

predicate_data = preprocess_dataset(X.astype(np.float32))

### Predicados e axiomas

Para a definição dos predicados, como informado anteriormente, foi feita a criação de modelos PyTorch, usando um modelo base e alterando o número de valores de entrada dependendo do número de variáveis usadas em cada predicado.

Foi também aqui que começaram os problemas com a implementação, visto que não consegui encontrar uma forma de definir os axiomas no contexto do LTNTorch. Eu acredito que seria possível alterando os predicados para compararem suas entradas de trens e carros com labels em vetores one-hot, como no exemplo dos caranguejos, mas com os vetores tendo tamanhos diferentes para acomodar os diferentes números de classes em cada atributo, mas eu não sei como poderia fazer isso no LTNTorch e os exemplos não tinham nada equivalente.

In [47]:
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())

In [36]:
# Define the predicates
class PredicateModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_dim, hidden_dim)
        self.layer2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = torch.nn.Linear(hidden_dim, 1)
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.elu(self.layer1(x))
        x = self.elu(self.layer2(x))
        return self.sigmoid(self.layer3(x))

# Define each predicate using a model
num_cars = ltn.Predicate(PredicateModel(2, 16))
num_loads = ltn.Predicate(PredicateModel(2, 16))
num_wheels = ltn.Predicate(PredicateModel(3, 16))
length = ltn.Predicate(PredicateModel(3, 16))
shape = ltn.Predicate(PredicateModel(3, 16))
num_car_loads = ltn.Predicate(PredicateModel(3, 16))
load_shape = ltn.Predicate(PredicateModel(3, 16))
direction = ltn.Predicate(PredicateModel(1, 16))

In [50]:
var1 = ltn.Variable("x", torch.tensor(predicate_data["num_cars"]))

In [51]:
var1.value[:, 0]

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
        28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
        42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55.,
        56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69.,
        70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82., 83.,
        84., 85., 86., 87., 88., 89., 90., 91., 92., 93., 94., 95., 96., 97.,
        98., 99.])

In [55]:
num_cars(var1)

LTNObject(value=tensor([0.4571, 0.4452, 0.4337, 0.4140, 0.4109, 0.3996, 0.3806, 0.3891, 0.3621,
        0.3694, 0.3453, 0.3369, 0.3393, 0.3202, 0.3182, 0.3075, 0.2948, 0.2861,
        0.2778, 0.2679, 0.2581, 0.2486, 0.2393, 0.2317, 0.2228, 0.2141, 0.2044,
        0.1964, 0.1881, 0.1819, 0.1745, 0.1666, 0.1604, 0.1537, 0.1468, 0.1411,
        0.1349, 0.1292, 0.1238, 0.1187, 0.1137, 0.1089, 0.1043, 0.0995, 0.0953,
        0.0911, 0.0873, 0.0838, 0.0802, 0.0762, 0.0731, 0.0697, 0.0672, 0.0639,
        0.0611, 0.0582, 0.0559, 0.0534, 0.0508, 0.0485, 0.0464, 0.0446, 0.0423,
        0.0410, 0.0389, 0.0369, 0.0355, 0.0337, 0.0326, 0.0309, 0.0293, 0.0282,
        0.0272, 0.0256, 0.0248, 0.0237, 0.0224, 0.0213, 0.0203, 0.0194, 0.0187,
        0.0177, 0.0172, 0.0162, 0.0154, 0.0148, 0.0140, 0.0135, 0.0129, 0.0123,
        0.0119, 0.0113, 0.0106, 0.0101, 0.0098, 0.0092, 0.0090, 0.0086, 0.0082,
        0.0077], grad_fn=<ViewBackward0>), free_vars=['x'])

In [66]:
t = ltn.Variable("t", torch.tensor(range(len(X))))
nc = ltn.Variable("nc", torch.tensor(np.array(predicate_data["num_cars"])[:, 1]))
dir = ltn.Variable("dir", torch.tensor(np.array(predicate_data["direction"])[:, 1]))

In [38]:
# Define axioms (examples)
axioms = [
    Forall(
        ltn.Variable("x", torch.tensor(predicate_data["num_cars"])) ,
        Implies(
            num_cars(["x[0]", "x[1]"]),
            direction(["x[0]"])
        )
    ),
    Forall(
        ltn.Variable("x", torch.tensor(predicate_data["shape"])),
        Implies(
            shape(["x[0]", "x[1]", "x[2]"]),
            direction(["x[0]"])
        )
    )
]

TypeError: Expected parameter 'inputs' to be a tuple of LTNObject, but got [<class 'list'>]

### Treino e teste

Como não fui capaz de definir direito os predicados e axiomas, não cheguei a implementar o treinamento, mas ainda escrevi um esqueleto do que teria se tornado o treino, acompanhando os parâmetros de cada subrede e agregando a satisfação dos axiomas.

In [ ]:
# Train the model
optimizer = torch.optim.Adam(
    list(num_cars.parameters()) +
    list(num_loads.parameters()) +
    list(num_wheels.parameters()) +
    list(length.parameters()) +
    list(shape.parameters()) +
    list(num_car_loads.parameters()) +
    list(load_shape.parameters()) +
    list(direction.parameters()),
    lr=0.001
)

for epoch in range(100):
    optimizer.zero_grad()
    sat_agg = ltn.fuzzy_ops.SatAgg()(*axioms)
    loss = 1. - sat_agg
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Satisfaction: {sat_agg.item()} Loss: {loss.item()}")

# Evaluate the model
print("Final Satisfaction Level:", sat_agg.item())

## Implementação incorreta

Antes de tentar a implemmentação com meta-rede, como o problema é tecnicamennte um de classificação binária (leste ou oeste), eu decidi verificar os resultados de um modelo binário na base de dados. Como mencionado anteriormente, este modelo foi baseado no exemplo de classificação binária do LTNTorch.

### Preparo da base

Na parte de preparar a base há uma célula comentada que corresponde a preparar o modelo para prever o cluster do trem, ao invés da direção. Isso dá resultados muito melhores e, embora não seja o objetivo do trabalho, implica que existe uma relação lógica mais fácil de se encontra entre os cluster do que entre as direções, o que não é muito surpreendente.

In [32]:
# X_tmp = X.drop(["Class_attribute"], axis=1)
# X_labels = X_tmp["Cluster"]
# X_tmp.drop(["Cluster"], axis=1, inplace=True)

In [33]:
X_tmp = X.drop(["Cluster"], axis=1)
X_labels = X_tmp["Class_attribute"]
X_tmp.drop(["Class_attribute"], axis=1, inplace=True)

In [34]:
X_treino = torch.from_numpy(X_tmp.to_numpy().astype(np.float32))
y_treino = torch.from_numpy(X_labels.to_numpy().astype(np.float32))

In [35]:
print(X_treino.shape)
print(y_treino.shape)

torch.Size([100, 32])
torch.Size([100])


### Predicados, conectivos e quantificadores

In [36]:
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())

Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Equiv = ltn.Connective(ltn.fuzzy_ops.Equiv(ltn.fuzzy_ops.AndProd(), ltn.fuzzy_ops.ImpliesReichenbach()))

Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=6), quantifier="e")

SatAgg = ltn.fuzzy_ops.SatAgg()

In [37]:
# we define predicate A
class ModelA(torch.nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()
        self.sigmoid = torch.nn.Sigmoid()
        # self.dropout = torch.nn.Dropout(0.2)
        self.layer1 = torch.nn.Linear(32, 16)
        self.layer2 = torch.nn.Linear(16, 16)
        self.layer3 = torch.nn.Linear(16, 1)
        self.elu = torch.nn.ELU()

    def forward(self, x):
        x = self.elu(self.layer1(x))
        # x = self.dropout(x)

        x = self.elu(self.layer2(x))
        # x = self.dropout(x)

        return self.sigmoid(self.layer3(x))


A = ltn.Predicate(ModelA())

### Funções de utilidade

In [43]:
# this is a standard PyTorch DataLoader to load the dataset for the training and testing of the model
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]

            yield data, labels

Métricas para a avaliação do modelo

In [44]:
# define metrics for evaluation of the model

# it computes the overall satisfaction level on the knowledge base using the given data loader (train or test)
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        x_A = ltn.Variable("x_A", data[torch.nonzero(labels)])  # positive examples
        x_not_A = ltn.Variable("x_not_A",
                               data[torch.nonzero(torch.logical_not(labels))])  # negative examples
        mean_sat += SatAgg(
            Forall(x_A, A(x_A)),
            Forall(x_not_A, Not(A(x_not_A)))
        )
    mean_sat /= len(loader)
    return mean_sat

In [45]:
# it computes the overall accuracy of the predictions of the trained model using the given data loader
# (train or test)
def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = A.model(data).detach().numpy()
        predictions = np.where(predictions > 0.5, 1., 0.).flatten()
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)

# create train and test loader, 50 points each
# batch size is 64, meaning there is only one batch for epoch
train_loader = DataLoader(X_treino[:50], y_treino[:50], 64, True)
test_loader = DataLoader(X_treino[50:], y_treino[50:], 64, False)

### Treino e teste

In [46]:
optimizer = torch.optim.Adam(A.parameters(), lr=0.001)

# training of the predicate A using a loss containing the satisfaction level of the knowledge base
# the objective it to maximize the satisfaction level of the knowledge base
for epoch in range(1000):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[torch.nonzero(labels)]) # positive examples
        x_not_A = ltn.Variable("x_not_A", data[torch.nonzero(torch.logical_not(labels))]) # negative examples
        sat_agg = SatAgg(
            Forall(x_A, A(x_A)),
            Forall(x_not_A, Not(A(x_not_A)))
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    if epoch % 20 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

ValueError: Input arrays use different devices: cpu, cpu

# **Questão 3: Discussão**

Como não fui capaz de finalizar o modelo, também não foi possível comparar com os resultados do livro. Com isso dito, eu acredito que os resultados teriam sido ruins, pois os clusters dos quais os axiomas foram derivados não separavam de forma significativa as direções dos trens, portanto, embora o modelo teria sido eficaz em prever o cluster a que o trem pertence, ele provavelmente não teria tido sucesso em prever a direção.

Quanto a extração de uma regra genérica de classificação do modelo, isso poderia ser feito através do TREPAN, um algoritmo que extrai árvores de decisão a partir de redes neurais, como a LTN usada neste trabalho.

O algoritmo funciona tratando uma rede neural como uma caixa-preta, onde insere uma entrada e verifica a saída. Esse mecanismo é integrado no processo de aprendizado de uma árvore de decisão, de tal forma que ao invés de aprender com um conjunto de exemplos, ela aprende fazendo perguntas a rede. No decorrer do aprendizado, o algoritmo faz splits de m-of-n, onde de n condições, m devem ser satisfeitas.

Dessa forma, o algoritmo cria uma árvore de decisão que aproxima os resultados da rede neural, servindo como uma forma mais legível de entender seu funcionamento.